In [1]:
#https://pysal.org/libpysal/notebooks/weights.html
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000) ## 800GB?

from libpysal.weights import Queen, Rook, KNN

from pysal.lib import weights
from pysal.explore import esda
from pysal.model import spreg

from shapely import wkb
from sklearn.cluster import KMeans

import matplotlib.pyplot as plt
import statsmodels.formula.api as sm

# import contextily
import geopandas as gpd
import numpy as np
import pandas as pd
import seaborn as sns

import branca
import folium
import shared_utils

from siuba import *

pd.set_option('display.max_columns', None) 

/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
/opt/conda/lib/python3.10/site-packages/spaghetti/network.py:39: FutureWarning: The next major release of pysal/spaghetti (2.0.0) will drop support for all ``libpysal.cg`` geometries. This change is a first step in refactoring ``spaghetti`` that is expected to result in dramatically reduced runtimes for network instantiation and operations. Users currently requiring network and point pattern input as ``libpysal.cg`` geometries should prepare for this simply by converting to ``shapely`` geometries.
  warnings.warn(f"{dep_msg}", FutureWarning)


In [6]:
gdf = gpd.read_parquet('gs://calitp-analytics-data/data-analyses/ahsc_grant/analytical_tbl.parquet')
gdf = gdf.dropna()
gdf

,calitp_itp_id,stop_id,geometry,stop_name,n_trips_weekday,n_trips_sat,n_trips_sun,n_routes_weekday,n_routes_sat,n_routes_sun,sat_ons,sun_ons,weekday_ons,sum_tracts,sum_total_pop,sum_households,sum_not_us_citizen_pop,sum_black_pop,sum_hispanic_pop,sum_youth_pop,sum_seniors_pop,sum_inc_extremelylow,sum_inc_verylow,sum_inc_low,sum_pop_determined_poverty_status,sum_poverty,sum_no_car,sum_no_cars,sum_land_area,sum_jobs,land_area_sqkm,pop_density,job_density,pct_not_us_citizen_pop,pct_black_pop,pct_hispanic_pop,pct_youth_pop,pct_seniors_pop,pct_inc_extremelylow,pct_inc_verylow,pct_inc_low,pct_poverty,pct_pop_workers_no_car,pct_hh_no_cars
0,293,423,POINT (25245.404 -397427.355),State & Alamar,60.0,56.0,55.0,3.0,2.0,2.0,567.0,566.0,4556.0,4.0,20153.0,9058.0,1370.0,180.0,3890.0,4776.0,4677.0,1124.0,1132.0,1703.0,19849.0,1676.0,115.0,448.0,8970055.0,12021.0,8.970055,2246.697484,1340.125562,0.067980,0.008932,0.193023,0.236987,0.232075,0.124089,0.124972,0.188011,0.084438,0.005706,0.049459
1,293,55,POINT (15628.312 -397010.130),Encina & Fairview,25.0,11.0,11.0,1.0,1.0,1.0,840.0,806.0,10303.0,2.0,7287.0,2572.0,644.0,133.0,1653.0,2109.0,1670.0,165.0,266.0,517.0,7287.0,336.0,34.0,61.0,6030586.0,2686.0,6.030586,1208.340284,445.396185,0.088377,0.018252,0.226842,0.289420,0.229175,0.064152,0.103421,0.201011,0.046110,0.004666,0.023717
2,293,239,POINT (23548.788 -398980.627),Veronica Springs & Torino,12.0,12.0,8.0,1.0,1.0,1.0,28.0,10.0,175.0,1.0,3455.0,1502.0,310.0,0.0,649.0,765.0,1064.0,141.0,186.0,262.0,3371.0,206.0,0.0,161.0,4129069.0,383.0,4.129069,836.750367,92.756987,0.089725,0.000000,0.187844,0.221418,0.307959,0.093875,0.123835,0.174434,0.061109,0.000000,0.107190
3,293,63,POINT (28266.459 -399563.850),Haley & Garden,25.0,21.0,14.0,1.0,1.0,1.0,148.0,85.0,978.0,2.0,10410.0,3823.0,1792.0,313.0,6028.0,3492.0,1260.0,920.0,611.0,741.0,10228.0,1877.0,189.0,499.0,4544564.0,23265.0,4.544564,2290.648784,5119.302974,0.172142,0.030067,0.579059,0.335447,0.121037,0.240649,0.159822,0.193827,0.183516,0.018156,0.130526
4,293,509,POINT (33779.852 -397621.940),None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13741,208,1619,POINT (-169553.548 -154399.663),None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13742,208,1623,POINT (-169757.062 -154073.881),None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13743,208,5102,POINT (-148711.025 -147857.530),None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13744,208,1616,POINT (-169831.216 -153784.483),None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
sns.histplot(gdf['weekday_ons'])

In [ ]:
sns.histplot(np.log(gdf['weekday_ons']))

In [ ]:
sns.histplot(gdf['pop_density'])

In [ ]:
sns.histplot(np.log(gdf['pop_density']))

In [ ]:
sns.histplot(np.log(gdf['job_density']))

In [ ]:
gdf.columns

In [8]:
# Multiply all percentage columns by 100 for easier OLS interpretation
for column in gdf.columns:
    if 'pct' in column:
        gdf[column] = gdf[column] * 100
gdf.head()

,calitp_itp_id,stop_id,geometry,stop_name,n_trips_weekday,n_trips_sat,n_trips_sun,n_routes_weekday,n_routes_sat,n_routes_sun,sat_ons,sun_ons,weekday_ons,sum_tracts,sum_total_pop,sum_households,sum_not_us_citizen_pop,sum_black_pop,sum_hispanic_pop,sum_youth_pop,sum_seniors_pop,sum_inc_extremelylow,sum_inc_verylow,sum_inc_low,sum_pop_determined_poverty_status,sum_poverty,sum_no_car,sum_no_cars,sum_land_area,sum_jobs,land_area_sqkm,pop_density,job_density,pct_not_us_citizen_pop,pct_black_pop,pct_hispanic_pop,pct_youth_pop,pct_seniors_pop,pct_inc_extremelylow,pct_inc_verylow,pct_inc_low,pct_poverty,pct_pop_workers_no_car,pct_hh_no_cars
0,293,423,POINT (25245.404 -397427.355),State & Alamar,60.0,56.0,55.0,3.0,2.0,2.0,567.0,566.0,4556.0,4.0,20153.0,9058.0,1370.0,180.0,3890.0,4776.0,4677.0,1124.0,1132.0,1703.0,19849.0,1676.0,115.0,448.0,8970055.0,12021.0,8.970055,2246.697484,1340.125562,6.797995,0.893167,19.302337,23.698705,23.207463,12.408920,12.497240,18.801060,8.443750,0.570635,4.945904
1,293,55,POINT (15628.312 -397010.130),Encina & Fairview,25.0,11.0,11.0,1.0,1.0,1.0,840.0,806.0,10303.0,2.0,7287.0,2572.0,644.0,133.0,1653.0,2109.0,1670.0,165.0,266.0,517.0,7287.0,336.0,34.0,61.0,6030586.0,2686.0,6.030586,1208.340284,445.396185,8.837656,1.825168,22.684232,28.941951,22.917524,6.415241,10.342146,20.101089,4.610951,0.466584,2.371695
2,293,239,POINT (23548.788 -398980.627),Veronica Springs & Torino,12.0,12.0,8.0,1.0,1.0,1.0,28.0,10.0,175.0,1.0,3455.0,1502.0,310.0,0.0,649.0,765.0,1064.0,141.0,186.0,262.0,3371.0,206.0,0.0,161.0,4129069.0,383.0,4.129069,836.750367,92.756987,8.972504,0.000000,18.784370,22.141823,30.795948,9.387483,12.383489,17.443409,6.110946,0.000000,10.719041
3,293,63,POINT (28266.459 -399563.850),Haley & Garden,25.0,21.0,14.0,1.0,1.0,1.0,148.0,85.0,978.0,2.0,10410.0,3823.0,1792.0,313.0,6028.0,3492.0,1260.0,920.0,611.0,741.0,10228.0,1877.0,189.0,499.0,4544564.0,23265.0,4.544564,2290.648784,5119.302974,17.214217,3.006724,57.905860,33.544669,12.103746,24.064871,15.982213,19.382684,18.351584,1.815562,13.052577
4,293,509,POINT (33779.852 -397621.940),None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# add constant to ridership to avoid log(0) - small bias but empirically supported
gdf['log_weekday_ons'] = np.log(gdf['weekday_ons']+1)
#gdf['log_weekday_ons'] = gdf['log_weekday_ons'].replace(np.NINF, 0) #replace undefined values produced from np.log(0) w/ 0

# also log pop and job density
gdf['log_pop_density'] = np.log(gdf['pop_density']+1)
gdf['log_job_density'] = np.log(gdf['job_density']+1)

In [10]:
# Weekday_ons ~ n_trips_weekday + pop_density + job_density + pct: 
# Intuition is to only use use pct variables because we don’t want to double count people in areas around multiple stops. Whereas, trips are characteristic of the stop itself

explanatory_vars = ['n_trips_weekday', 'n_routes_weekday','pop_density', 'job_density','pct_not_us_citizen_pop',
                    'pct_youth_pop', 'pct_seniors_pop', 'pct_pop_workers_no_car', 'pct_poverty']

explanatory_vars_short = ['n_trips_weekday','n_routes_weekday','pop_density', 'pct_not_us_citizen_pop',
                    'pct_youth_pop', 'pct_seniors_pop', 'pct_pop_workers_no_car']


Comparing version with job_density and poverty to one without, to evaluate multicollinearity. Had race variables, but dropped them due to lack of correlation with ridership.

### OLS Untransformed model

In [11]:
# model with more explanatory variables
m = spreg.OLS(gdf[['weekday_ons']].values, gdf[explanatory_vars].values, 
                  name_y = 'weekday_ons', name_x = explanatory_vars)
print(m.summary)

Exception: one or more input arrays have missing/NaN values

In [ ]:
# model with fewer explanatory variables
m2 = spreg.OLS(gdf[['weekday_ons']].values, gdf[explanatory_vars_short].values, 
                  name_y = 'weekday_ons', name_x = explanatory_vars_short)
print(m2.summary)

### OLS Log-Linear model

In [ ]:
gdf['log_weekday_ons'].replace(np.NINF, 0).value_counts()

In [ ]:
# log model with more vars
m_log = spreg.OLS(gdf[['log_weekday_ons']].values, gdf[explanatory_vars].values, 
                  name_y = 'log_weekday_ons', name_x = explanatory_vars)
print(m_log.summary)

In [ ]:
# log model with fewer vars
m_log2 = spreg.OLS(gdf[['log_weekday_ons']].values, gdf[explanatory_vars_short].values, 
                  name_y = 'log_weekday_ons', name_x = explanatory_vars_short)
print(m_log2.summary)

In [ ]:
#exponentiate coefficients to interpret
import math

m_log_df = pd.DataFrame() 
m_log_df['Variable']=m_log.name_x
m_log_df['Coefficient']=m_log.betas
m_log_df.dtypes

In [ ]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)

m_log_df['Coeff_Exp']=(np.exp(m_log_df['Coefficient'])-1)*100
m_log_df

In [ ]:
# testing ons-per-capita - doesn't work
gdf = (gdf
       >> mutate(weekday_ons_per_capita = _.weekday_ons/_.sum_total_pop)
      )

In [ ]:
m_log = spreg.OLS(gdf[['weekday_ons_per_capita']].values, gdf[explanatory_vars].values, 
                  name_y = 'weekday_ons_per_capita', name_x = explanatory_vars)
print(m_log.summary)

## Spatial Weights

In [ ]:
# 3 nearest neighbors
knn = weights.KNN.from_dataframe(gdf, k=3)

In [ ]:
# chart indicates that errors are spatially dependent
lag_residual = weights.spatial_lag.lag_spatial(knn, m.u)
ax = sns.regplot(
    m.u.flatten(),
    lag_residual.flatten(),
    line_kws=dict(color="orangered"),
    ci=None,
)
ax.set_xlabel("Model Residuals - $u$")
ax.set_ylabel("Spatial Lag of Model (1) Residuals - $W u$");

In [ ]:
lag_residual = weights.spatial_lag.lag_spatial(knn, m_log.u)
ax = sns.regplot(
    m_log.u.flatten(),
    lag_residual.flatten(),
    line_kws=dict(color="orangered"),
    ci=None,
)
ax.set_xlabel("Model Residuals - $u$")
ax.set_ylabel("Spatial Lag of Log Model Residuals - $W u$");

From context of https://geographicdata.science/book/notebooks/11_regression.html

In [ ]:
# Re-weight W to 3 nearest neighbors
knn.reweight(k=3, inplace=True)
# Row standardise weights
knn.transform = "R"

In [ ]:
# Run LISA on residuals
outliers = esda.moran.Moran_Local(m_log.u, knn, permutations=9999)
# Select only LISA cluster cores
error_clusters = outliers.q % 2 == 1
# Filter out non-significant clusters
error_clusters &= outliers.p_sim <= 0.001
# Add `error_clusters` and `local_I` columns
ax = (
    gdf.assign(
        error_clusters=error_clusters,
        local_I=outliers.Is
        # Retain error clusters only
    )
    .query(
        "error_clusters"
        # Sort by I value to largest plot on top
    )
    .sort_values(
        "local_I"
        # Plot I values
    )
    .plot("local_I", cmap="bwr", marker=".")
)
# Add basemap
contextily.add_basemap(ax, crs=gdf.crs)
# Remove axes
ax.set_axis_off();

And then fit the model using the OLS class in Pysal's `spreg`:

In [ ]:
# Fit spatial error model with `spreg`
# (GMM estimation allowing for heteroskedasticity)
m_knn_err = spreg.GM_Error_Het(
    # Dependent variable
    gdf[["log_weekday_ons"]].values,
    # Independent variables
    gdf[explanatory_vars].values,
    # Spatial weights matrix
    w=knn,#K=3
    # Dependent variable name
    name_y="log_weekday_ons",
    # Independent variables names
    name_x=explanatory_vars,
)
print(m_knn_err.summary)

In [36]:
# Fit spatial lag model with `spreg`
# (GMM estimation)
m_knn = spreg.GM_Lag(
    # Dependent variable
    gdf[["log_weekday_ons"]].values,
    # Independent variables
    gdf[explanatory_vars].values,
    # Spatial weights matrix
    w=knn,#K=3
    # Dependent variable name
    name_y="log_weekday_ons",
    # Independent variables names
    name_x=explanatory_vars,
)
print(m_knn.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: SPATIAL TWO STAGE LEAST SQUARES
--------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :log_weekday_ons                Number of Observations:       13723
Mean dependent var  :      7.5420                Number of Variables   :          10
S.D. dependent var  :      2.2350                Degrees of Freedom    :       13713
Pseudo R-squared    :      0.5050
Spatial Pseudo R-squared:  0.4074

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT       4.2757066       0.1964463      21.7652675       0.0000000
     n_trips_weekday       0.0154517       0.0004115      37.5534305       0.0000000
         pop_density       0.0000986       0.0000076    

In [37]:
#exponentiate coefficients to interpret
import math

m_knn_df = pd.DataFrame() 
m_knn_df['Variable']=m_knn.name_x
m_knn_df['Coefficient']=m_knn.betas
m_knn_df.dtypes

ValueError: Length of values (10) does not match length of index (9)

Stop here: below spatial weights exploration is for polygons
To do: fixed distance weights maybe with distance decay?

In [3]:
gdf.crs

NameError: name 'gdf' is not defined

In [ ]:

w_dist = knn = weights.DistanceBand.from_dataframe(gdf, k=3)

In [ ]:
ax = gdf.plot()
ax.set_axis_off()

In [ ]:
#Rook neighbors are those states that share an edge on their respective borders:
#This shouldn't translate to stops since there aren't polygons/borders, but points?
w_rook = Rook.from_dataframe(gdf)
w_rook.n

In [ ]:
w_rook.pct_nonzero

In [ ]:
# Fit spatial lag model with `spreg`
# (GMM estimation)
m_rook = spreg.GM_Lag(
    # Dependent variable
    gdf[["log_weekday_ons"]].values,
    # Independent variables
    gdf[explanatory_vars].values,
    # Spatial weights matrix
    w=w_rook,
    # Dependent variable name
    name_y="log_weekday_ons",
    # Independent variables names
    name_x=explanatory_vars,
)
print(m_rook.summary)

In [ ]:
w_queen = Queen.from_dataframe(gdf)
w_queen.n

In [ ]:
m_queen = spreg.GM_Lag(
    # Dependent variable
    gdf[["log_weekday_ons"]].values,
    # Independent variables
    gdf[explanatory_vars].values,
    # Spatial weights matrix
    w=w_queen,
    # Dependent variable name
    name_y="log_weekday_ons",
    # Independent variables names
    name_x=explanatory_vars,
)
print(m_queen.summary)